In [8]:
import os
import pandas as pd
from docx import Document
from PyPDF2 import PdfReader

def extract_text_from_files(folder_path):
    extracted_data = {}

    for filename in os.listdir(folder_path):
        filepath = os.path.join(folder_path, filename)
        text = ""

        if filename.lower().endswith(('.xlsx', '.xls', '.xlsm')):
            try:
                xls = pd.ExcelFile(filepath)
                sheets_text = []
                for sheet in xls.sheet_names:
                    df = xls.parse(sheet)
                    sheet_text = f"Sheet: {sheet}\n{df.to_string(index=False)}"
                    sheets_text.append(sheet_text)
                text = "\n\n".join(sheets_text)
            except Exception as e:
                text = f"Error reading Excel file: {e}"

        else:
            continue  # skip unsupported file types

        extracted_data[filename] = text

    return extracted_data

folder_path = r"C:\Users\salsubhi1\PycharmProjects\Enigmatic Research\Dr.X Files"
results = extract_text_from_files(folder_path)

# Print one example
for file, content in results.items():
    print(f"\n=== {file} ===\n")
    print(content[:3000])  # show first 3000 characters
    #break  # remove this if you want to print all files


=== Loan amortisation schedule1.xlsx ===

Sheet: Loan Schedule
 Unnamed: 0                  Unnamed: 1                 Unnamed: 2         Unnamed: 3          Unnamed: 4     Unnamed: 5                   Unnamed: 6  Unnamed: 7     Unnamed: 8      Unnamed: 9          Unnamed: 10
        NaN                         NaN Loan Amortisation Schedule                NaN                 NaN            NaN                          NaN         NaN            NaN             NaN                  NaN
        NaN                         NaN                        NaN                NaN                 NaN            NaN                          NaN         NaN            NaN             NaN                  NaN
        NaN                Enter Values                        NaN                NaN                 NaN            NaN                 Loan Summary         NaN            NaN             NaN                  NaN
        NaN                 Loan amount                        NaN              

C:\Users\salsubhi1\Anaconda3\envs\geo_env\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
C:\Users\salsubhi1\Anaconda3\envs\geo_env\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
C:\Users\salsubhi1\Anaconda3\envs\geo_env\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\salsubhi1\Anaconda3\envs\geo_env\lib\site-packages\openpyxl\reader\workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'Wedding budget'!$A:$K.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


In [14]:
import os
import pandas as pd
import tiktoken
from docx import Document
from PyPDF2 import PdfReader

# Set tokenizer and chunk size
tokenizer = tiktoken.get_encoding("cl100k_base")
max_tokens = 500  # adjust as needed

def chunk_text(text, chunk_size=max_tokens):
    tokens = tokenizer.encode(text)
    chunks = []
    for i in range(0, len(tokens), chunk_size):
        chunk_tokens = tokens[i:i + chunk_size]
        chunk_text = tokenizer.decode(chunk_tokens)
        chunks.append(chunk_text)
    return chunks

def extract_text_from_files(folder_path):
    extracted_chunks = []

    for filename in os.listdir(folder_path):
        filepath = os.path.join(folder_path, filename)
        text = ""

        if filename.lower().endswith(('.xlsx', '.xls', '.xlsm')):
            try:
                xls = pd.ExcelFile(filepath)
                sheets_text = []
                for sheet in xls.sheet_names:
                    df = xls.parse(sheet)
                    sheet_text = f"Sheet: {sheet}\n{df.to_string(index=False)}"
                    sheets_text.append(sheet_text)
                text = "\n\n".join(sheets_text)
            except Exception as e:
                text = f"Error reading Excel file: {e}"
        else:
            continue  # skip unsupported file types

        # Tokenize and chunk the extracted text
        chunks = chunk_text(text)

        for idx, chunk in enumerate(chunks, start=1):
            extracted_chunks.append({
                "source": filename,
                "page_number": 1,  # All Excel files are treated as one "page"
                "chunk_number": idx,
                "text": chunk
            })

    return extracted_chunks

# Folder path
folder_path = r"C:\Users\salsubhi1\PycharmProjects\Enigmatic Research\Dr.X Files"

# Run extraction and chunking
chunked_data = extract_text_from_files(folder_path)

# Convert to DataFrame
df = pd.DataFrame(chunked_data)

# Show sample output
print(df.head())

# Save to CSV
df.to_csv("excel_chunks_text.csv", index=False)


                             source  page_number  chunk_number   
0  Loan amortisation schedule1.xlsx            1             1  \
1  Loan amortisation schedule1.xlsx            1             2   
2  Loan amortisation schedule1.xlsx            1             3   
3                Loan analysis.xlsx            1             1   
4                Loan analysis.xlsx            1             2   

                                                text  
0  Sheet: Loan Schedule\n Unnamed: 0             ...  
1  00:00        4490.917146          425.749521  ...  
2  2.931682      356.686841             88.43253\...  
3  Sheet: LOAN ANALYSIS\n Unnamed: 0 LOAN ANALYSI...  
4     39.512090          NaN                     ...  


C:\Users\salsubhi1\Anaconda3\envs\geo_env\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
C:\Users\salsubhi1\Anaconda3\envs\geo_env\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
C:\Users\salsubhi1\Anaconda3\envs\geo_env\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\salsubhi1\Anaconda3\envs\geo_env\lib\site-packages\openpyxl\reader\workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'Wedding budget'!$A:$K.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
